In [1]:
import os
import pyautogui
import urllib.parse
import time
import pyperclip
import quopri
from bs4 import BeautifulSoup
from email import policy
from email.parser import BytesParser
import requests
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import pickle

# 1.Data Collection

In [2]:
# Create an automatic process to download the job search pages in Linkedin
def auto_download_search(url,file_name):
    pyautogui.hotkey('ctrl', '2') # switch Google Chrome tabs
    time.sleep(1)
    
    pyautogui.click(207,55) # click the browser's address bar.
    time.sleep(0.5)
    
    pyautogui.hotkey('ctrl', 'a') # select all content in the address bar.
    pyperclip.copy(url) # copy the url
    time.sleep(0.5)
    
    pyautogui.hotkey('ctrl', 'v') # paste the url
    time.sleep(0.5)
    
    pyautogui.hotkey('enter') # access the url
    time.sleep(10)
    
    pyautogui.click(882,1023)
    pyautogui.mouseDown() # scroll down the page to look through all job posts
    time.sleep(10)
    pyautogui.mouseUp()# release the left mouse button
    
    pyautogui.hotkey('ctrl', 's') # save the page to local path
    time.sleep(2)
    pyperclip.copy(file_name) # copy the file name
    pyautogui.hotkey('ctrl', 'v') # paste the file name
    time.sleep(2)
    pyautogui.hotkey('enter') # save the page with given name

In [3]:
# create a process to look through all the pages of search results
def search_page_download():
    for i in range(0,1000,25):
        url = f"https://www.linkedin.com/jobs/search/?currentJobId=3733476715&f_SB2=1&f_TPR=r2592000&geoId=103644278&keywords=data%20scientist&location=United%20States&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&sortBy=R&spellCorrectionEnabled=true&start={i}"
        file_name = f"data_science_job_{int(i/25)}.mhtml"
        auto_download_search(url, file_name)

In [4]:
# download all pages to collect 1000 job posts related to data science
search_page_download()

In [3]:
# extract urls of job posts from all the pages
def job_url(file_path):
    urls = []
    # Open the MHTML file in binary mode and parse it
    with open(file_path, 'rb') as file:
        msg = BytesParser(policy=policy.default).parse(file)

    # Decode the HTML part correctly
    html_part = None
    for part in msg.walk():
        content_type = part.get_content_type()
        if content_type == 'text/html':
            html_part = part.get_payload(decode=True)
            break
    
    # parse the HTML
    charset = 'utf-8'
    decoded_html = html_part.decode(charset)
    soup = BeautifulSoup(decoded_html, 'html.parser')
    jobs = soup.find(class_ = "scaffold-layout__list-container")
    job_info = jobs.find_all(class_ = "disabled ember-view job-card-container__link job-card-list__title")
    # extract the url for each job post
    for i in job_info:
        url = i['href']
        urls.append(url)
    return urls

In [4]:
job_urls = {}
i = 0
path = "D:/Study Abroad/course/DSCI441/project/webpages"

# get the urls of all job posts from the local job pages
for root, dirs, files in os.walk(path):
    for file in files:
        file_path = os.path.join(root, file)
        urls = job_url(file_path)
        for url in urls:
            job_urls[i] = {}
            job_urls[i]["url"] = url
            job_urls[i]["status"] = 0
            i += 1

In [40]:
# manually login the linkedin website
driver = webdriver.Chrome()
driver.get("https://www.linkedin.com/home")

# wait enough time to log in manually
time.sleep(20)  

# save the cookies to a file after login
pickle.dump(driver.get_cookies(), open("D:/Study Abroad/course/DSCI441/project/cookies.pkl", "wb"))

driver.quit()

In [14]:
# use selenium webdriver to iterate all the job pages
# set the options
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# load the previously saved cookies
cookies = pickle.load(open("D:/Study Abroad/course/DSCI441/project/cookies.pkl", "rb"))

# iterate all job post pages and download them to local path
for i in range(0,1000):
    index = i
    value = job_urls[index]
    
    # open a new driver after iterating the job post pages for 20 times in order to save memory
    if index % 20 == 0:
        driver = webdriver.Chrome(options=options)
        driver.get("https://www.linkedin.com/home")
        time.sleep(4)
        
        # load the cookies to the driver
        for cookie in cookies:
            driver.add_cookie(cookie)
        time.sleep(1)
    
    # get the url for job post
    url = value["url"]
    driver.get(url)
    
    time.sleep(5)
    
    # use the aria-label to locate the button
    button_aria_label = "Click to see more description"
    try:
        # Wait up to 10 seconds for the button to be clickable
        button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//button[@aria-label='{button_aria_label}']")))
        # click the button "Show More" to show the entire job description
        button.click()
        
        time.sleep(2) # wait up for all information displayed
        
    except Exception as e:
        break # if failed, jump to the next iteration
    
    pyautogui.hotkey('ctrl', 's') # save the page to local path
    time.sleep(2)
    
    file_name = "data_scientist_job_post_" + str(index)
    pyperclip.copy(file_name) # copy the file name
    
    pyautogui.hotkey('ctrl', 'v') # paste the file name
    time.sleep(2)
    
    if index % 20 == 0:
        pyautogui.hotkey('tab')  # select the save type
        time.sleep(1)

        pyautogui.hotkey('down')
        time.sleep(1)

        pyautogui.hotkey('up')  # save file to mhtml
        time.sleep(1)

        pyautogui.hotkey('enter') # confirm the file tyep
        time.sleep(1)
    
    pyautogui.hotkey('enter') # save
    time.sleep(np.random.randint(1,4)) # wait for random time
    
    if index % 20 == 19:
        driver.quit()